In [ ]:
import numpy as np
import pandas as pd
import time
import os
import tensorflow as tf
tf.config.run_functions_eagerly(False)
from plotting_funcs import *

#from VaDER.vader import VADER
#save_path = os.path.join('test_vader', 'vader.ckpt')
#np.random.seed(123)

In [ ]:
X = np.load('../DL-WORKSHOP/npy_files/features_All12.npy')
y = np.load('../DL-WORKSHOP/npy_files/labels_All12.npy')
y_letter = np.array([well[:1] for well in y])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labs = le.fit_transform(y)
labs_by_letter = le.fit_transform(y_letter)
print(len(X),len(y))
X_train, y_train = X, labs

In [ ]:
pd.Series(y_letter).value_counts()

In [ ]:
pd.Series(y).value_counts()

In [ ]:
# normalize (better for fitting)
for i in np.arange(X_train.shape[2]):
    X_train[:,:,i] = (X_train[:,:,i] - np.mean(X_train[:,:,i])) / np.std(X_train[:,:,i])

In [ ]:
W_train = None
vader = VADER(X_train=X_train, W_train=W_train, save_path=save_path, n_hidden=[60,32,4], k=3,
              learning_rate=1e-3, output_activation=None, recurrent=True, cell_type="GRU", batch_size=64)
# pre-train without latent loss
start = time.time()
vader.pre_fit(n_epoch=64, verbose=True)
# train with latent loss
vader.fit(n_epoch=64, verbose=True)
end = time.time()
print("Elapsed: ", end - start)

In [ ]:
# get the clusters
c = vader.cluster(X_train)
# get the re-constructions
p = vader.predict(X_train)
# compute the loss given the network
l = vader.get_loss(X_train)

In [ ]:
print(pd.Series(c).value_counts())

In [ ]:
latent = vader.map_to_latent(X_train)

In [ ]:
hex, hex_dict = plot_clustering(latent,labs_by_letter)

## specific well/gene group clustering

In [ ]:
X_G4 = np.array([x for x,well in zip(X,y) if well == "G4"])
specific_c = vader.cluster(X_G4)
specific_latent = vader.map_to_latent(X_G4)
print(pd.Series(specific_c).value_counts())

In [ ]:
X_F4 = np.array([x for x,well in zip(X,y) if well == "E6"])
specific_c2 = vader.cluster(X_F4)
specific_latent2 = vader.map_to_latent(X_F4)
print(pd.Series(specific_c2).value_counts())

In [ ]:
plot_dual_clustering(specific_latent,specific_c,specific_latent2,specific_c2,hex)

## transformer 

In [ ]:
vader = VADER(X_train=X_train, W_train=W_train, save_path=save_path, n_hidden=[64,4], k=3,
              learning_rate=1e-3, output_activation=None, recurrent=True, cell_type="Transformer", batch_size=64,
              cell_params={'d_model': 4, 'num_layers': 1, 'num_heads': 1, 'dff': 16, 'rate': 0.0})
# pre-train without latent loss
start = time.time()
vader.pre_fit(n_epoch=50, verbose=True)
# train with latent loss
vader.fit(n_epoch=50, verbose=True)
end = time.time()
print("Elapsed: ", end - start)